In [3]:
%pip install cv2

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement cv2 (from versions: none)
ERROR: No matching distribution found for cv2

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import cv2
import requests
import numpy as np
import io

# URL of your FastAPI endpoint
API_URL = 'http://127.0.0.1:8000/predict'

# Function to preprocess image
def preprocess_image(image):
    image = cv2.resize(image, (224, 224))
    image = np.transpose(image, (2, 0, 1)).astype(np.float32) / 255.0
    return image

# Function to make prediction using FastAPI endpoint
def predict_sign_language(image):
    # Convert image to JPEG format
    _, img_encoded = cv2.imencode('.jpg', image)
    
    # Send image as bytes
    response = requests.post(API_URL, files={'file': io.BytesIO(img_encoded.tobytes())})
    
    # Get prediction from response
    prediction = response.json()
    return prediction['predicted_class']

# Start video capture
cap = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess image
    processed_frame = preprocess_image(frame)

    # Print shape of processed frame for debugging
    print(processed_frame.shape)

    # Make prediction
    predicted_class = predict_sign_language(processed_frame)

    # Display predicted class on frame
    cv2.putText(frame, f'Predicted Class: {predicted_class}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the resulting frame
    cv2.imshow('Sign Language Alphabet Detection', frame)

    # Check for 'q' key press to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture and close OpenCV windows
cap.release()
cv2.destroyAllWindows()


(3, 224, 224)


error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:1124: error: (-215:Assertion failed) channels == 1 || channels == 3 || channels == 4 in function 'cv::imencode'


tensor flow model 


In [1]:
import cv2
print(cv2.getBuildInformation())



General configuration for OpenCV 4.9.0 =====================================
  Version control:               4.9.0

  Platform:
    Timestamp:                   2023-12-31T11:21:18Z
    Host:                        Windows 10.0.17763 AMD64
    CMake:                       3.24.2
    CMake generator:             Visual Studio 14 2015
    CMake build tool:            MSBuild.exe
    MSVC:                        1900
    Configuration:               Debug Release

  CPU/HW features:
    Baseline:                    SSE SSE2 SSE3
      requested:                 SSE3
    Dispatched code generation:  SSE4_1 SSE4_2 FP16 AVX AVX2
      requested:                 SSE4_1 SSE4_2 AVX FP16 AVX2 AVX512_SKX
      SSE4_1 (16 files):         + SSSE3 SSE4_1
      SSE4_2 (1 files):          + SSSE3 SSE4_1 POPCNT SSE4_2
      FP16 (0 files):            + SSSE3 SSE4_1 POPCNT SSE4_2 FP16 AVX
      AVX (8 files):             + SSSE3 SSE4_1 POPCNT SSE4_2 AVX
      AVX2 (36 files):           + SSSE3 SSE4_1 

In [8]:
import tensorflow as tf
from keras.models import load_model 

model = load_model("model.pkl")

converter = tf.lite.TFLiteConverter.from_keras_model(model)

converter.optimizations = [tf.lite.Optimizations.DEFAULT]

lite_model = converter.convert()

with open("lite_model.tflite", "wb") as f:
    f.write(lite_model)

OSError: Unable to open file (file signature not found)